In [1]:
import pandas as pd
# import konlpy 
from konlpy.tag import Kkma
# from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [2]:
stopwords = pd.read_csv('Korean_stopwords100.txt', sep="\t", header=None)
stopwords.columns = ["stopword", "POS", "ratio"]

In [3]:
stopwords_list = list(stopwords.stopword)

In [123]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
#         self.twitter = Twitter()
        self.stopwords = stopwords_list + ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
    ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "존경", "여러분", '올해', '지난해', '국가', '정부'] 
    # 정부? 국민? 
        
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''

        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.kkma.nouns(str(sentence)) # kkma로 바꿈. 
                if noun not in self.stopwords and len(noun) > 1]))
    
        return nouns

In [124]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    
    def build_sent_graph(self, sentence): # sen = ["안녕 나 이름 문재인", "신년 정부 인사 국민 문재인", "정부 박근혜 촛불 나 이름"] 와 같이 명사로 tokenize된 문장들의  iterable한 리스트를 받는다. 
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [125]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
    
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    
        return {idx: r[0] for idx, r in enumerate(ranks)}


In [126]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        self.sentences = self.sent_tokenize.text2sentences(text) # 문장들. 
    
        self.nouns = self.sent_tokenize.get_nouns(self.sentences) # nouns 말뭉치 문장들. 

        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def summarize(self, sent_num=3):
        summary = []
        index=[]
        
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
#         index.sort() # ㄱㄴㄷ 순으로 해놓는건 말도 안된다. 중요도 순으로 오도록 재설정. 
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
        
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
#         sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        ###
        sorted_rank_idx = []
        for k, v in rank_idx.items():
            sorted_rank_idx.append((k,v))
        sorted_rank_idx = sorted(sorted_rank_idx, key=lambda i: i[1], reverse=True)
        ###
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        
        #index.sort()
        for idx in index:
            keywords.append((self.idx2word[idx[0]], idx[1]))
        
        return keywords

In [127]:
president_2019_newyear = open('president2019newyear.txt',encoding='UTF8').read()

In [128]:
textrank = TextRank(president_2019_newyear)
for row in textrank.summarize(5):
    print(row)
    print()
print('keywords :',textrank.keywords())


한반도 비핵화에 대한 약속이 지켜 지고 평화가 완전히 제도화될 때까지 긴장을 늦추지 않겠습니다.

이러한 정책을 통해 지난해, 전반적인 가계 실질소득을 늘리고 의료, 보육, 통신 등의 필수 생계비를 줄일 수 있었습니다.

공정경제 법안의 조속한 입법을 위해 여야 정 국정 상설 협의체를 더욱 활성화 하도록 노력하겠습니다.

우리 정부가 추진하고 있는 ' 사람 중심 경제' 와 ' 혁신적 포용국가' 가 바로 그것입니다.

그래서 OECD, IMF 같은 국제기구와 주요 국가들은 ' 포용적 성장' 을 그 해법으로 제시하고 있습니다.

keywords : [('경제', 3.4950155210541545), ('성장', 2.8436652910569165), ('국민', 2.8429200618529946), ('혁신', 2.60634448771167), ('평화', 2.3451490969635), ('작년', 2.285739487031945), ('확대', 2.2504597249831635), ('지원', 2.159854221668958), ('추진', 2.119335346462586), ('문화', 2.039730659197367)]


In [129]:
president_2018_newyear = open('newyear_2018.txt',encoding='UTF8').read()

In [130]:
textrank = TextRank(president_2018_newyear)
for row in textrank.summarize(5):
    print(row)
    print()
print('keywords :',textrank.keywords())


국민의 삶을 개선하기 위해 정부도 혁신하겠습니다.

존경하는 국민 여러분, 한반도의 평화 정착으로 국민의 삶이 평화롭고 안정되어야 합니다.

국민들께 서는 자신의 소중한 일상을 국가에 내 어 주었습니다.

국민 개개인의 삶 속에 깊이 파고든 불안과 불신을 걷어내겠습니다.

남북이 공동으로 선언한 한반도 비핵화가 결코 양보할 수 없는 우리의 기본 입장입니다.

keywords : [('국민', 4.833902265835707), ('지원', 2.5210481766457775), ('목표', 2.0594848573016944), ('경제', 1.9911990299032991), ('안전', 1.9806093680379342), ('일자리', 1.9624204103610885), ('강화', 1.9510289888587673), ('평화', 1.9216194946682468), ('청년', 1.8943218529486812), ('민국', 1.8665510148973445)]
